# **Mediapipe To Mixamo**

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import json
from IPython.display import clear_output
import matplotlib.animation
from mediapipe_pose_to_json import mediapipe_helper as mp_helper
import mixamo_helper as mm_helper
import ntpath
import glob
import math

In [8]:
g_mixamoName_mediapipeName_map = mm_helper.get_mixamo_name_mediapipe_name_map()
g_mediapipeName_idx_map = mp_helper.get_name_idx_map()
g_mixamoName = mm_helper.mixamo_names

## **Mediapipe To Mixamo**

In [9]:
def avg3D(point1, point2):
    return [float(point1[0] + point2[0])/2.0, 
            float(point1[1]+ point2[1])/2.0, 
            float(point1[2]+ point2[2])/2.0, 
            float(point1[3]+ point2[3])/2.0]

def avgMixamoData(mediapipeJsonObject, frameNum, name1, name2):
    left = mediapipeJsonObject[frameNum]["keypoints3D"][g_mediapipeName_idx_map[name1]]
    right = mediapipeJsonObject[frameNum]["keypoints3D"][g_mediapipeName_idx_map[name2]]
    x = float(left['x'] + right['x']) / 2.0
    y = float(left['y'] + right['y']) / 2.0
    z = float(left['z'] + right['z']) / 2.0
    score = (left['score'] + right['score']) / 2.0 
    return [x, y, z, score]

def mediapipeToMixamoHip(resultKeyPointsJsonObject, mediapipeJsonObject, frameNum):
    #left hip <->right hip
    x, y, z, score = avgMixamoData(mediapipeJsonObject, frameNum, "left_hip", "right_hip")
    resultKeyPointsJsonObject["keypoints3D"].append({'x': 0.0, 
                                                     'y': 0.0, 
                                                     'z': 0.0, 
                                                     'score': score, 
                                                     'name': g_mixamoName[0][0], 
                                                     'parent': g_mixamoName[0][2]})
    lefthip2d = mediapipeJsonObject[frameNum]["keypoints"][g_mediapipeName_idx_map["left_hip"]]
    righthip2d = mediapipeJsonObject[frameNum]["keypoints"][g_mediapipeName_idx_map["right_hip"]]
    resultKeyPointsJsonObject["keypoints"].append({'x': (lefthip2d['x'] + righthip2d['x'])/ 2.0, 
                                                     'y': (lefthip2d['y']+ righthip2d['y'])/ 2.0, 
                                                     'z': (lefthip2d['z'] + righthip2d['z'])/ 2.0, 
                                                     'score':(lefthip2d['score']+ righthip2d['score'])/ 2.0, 
                                                     'name': g_mixamoName[0][0], 
                                                     'parent': g_mixamoName[0][2]})
    return
def mediapipeToMixamoSpine(resultKeyPointsJsonObject, mediapipeJsonObject, frameNum):
    # left_hip, right_hip, left_shoulder, right_shoulder
    hip = avgMixamoData(mediapipeJsonObject, frameNum, "left_hip", "right_hip")
    neck = avgMixamoData(mediapipeJsonObject, frameNum, "left_shoulder", "right_shoulder")
    spine2 = avg3D(hip, neck)
    spine1 = avg3D(hip, spine2)
    spine3 = avg3D(spine2, neck)
    spine = [spine1, spine2, spine3]
    for idx in range(0, len(spine)):
        resultKeyPointsJsonObject["keypoints3D"].append({'x': spine[idx][0], 
                                                     'y': spine[idx][1], 
                                                     'z': spine[idx][2], 
                                                     'score': spine[idx][3], 
                                                     'name': g_mixamoName[idx+1][0], 
                                                     'parent': g_mixamoName[idx+1][2]}) 

def mediapipeToMixamoNeck(resultKeyPointsJsonObject, mediapipeJsonObject, frameNum):
    # left_shoulder <-> right_shoulder
    x, y, z, score = avgMixamoData(mediapipeJsonObject, frameNum, "left_shoulder", "right_shoulder")
    resultKeyPointsJsonObject["keypoints3D"].append({'x': x, 
                                                     'y': y, 
                                                     'z': z, 
                                                     'score': score, 
                                                     'name': g_mixamoName[4][0], 
                                                     'parent': g_mixamoName[4][2]})
    return
def mediapipeToMixamoHead(resultKeyPointsJsonObject, mediapipeJsonObject, frameNum):
    # left_shoulder <-> right_shoulder
    x, y, z, score = avgMixamoData(mediapipeJsonObject, frameNum, "left_ear", "right_ear")
    resultKeyPointsJsonObject["keypoints3D"].append({'x': x, 
                                                     'y': y, 
                                                     'z': z, 
                                                     'score': score, 
                                                     'name': g_mixamoName[5][0], 
                                                     'parent': g_mixamoName[5][2]})
    return
def mediapipeToMixamoAll(resultKeyPointsJsonObject, mediapipeJsonObject, frameNum):
    frameKeypoints =  mediapipeJsonObject[frameNum]["keypoints3D"]
    for idx in range(6, len(g_mixamoName)):
        mediapipeName = g_mixamoName_mediapipeName_map[g_mixamoName[idx][0]]
        mediapipeIdx = g_mediapipeName_idx_map[mediapipeName]
        keypoint = frameKeypoints[mediapipeIdx]
        resultKeyPointsJsonObject["keypoints3D"].append({'x': keypoint['x'], 
                                                         'y': keypoint['y'], 
                                                         'z': keypoint['z'], 
                                                         'score': keypoint['score'], 
                                                         'name': g_mixamoName[idx][0], 
                                                         'parent': g_mixamoName[idx][2]})
    return

def mediapipeToMixamo(mediapipeJsonObject):
    jsonObject = {
        "fileName": mediapipeJsonObject['fileName'],
        "duration": mediapipeJsonObject['duration'],
        "ticksPerSecond": mediapipeJsonObject['ticksPerSecond'],
        "frames": []
    }
    for fidx in range(0, len(mediapipeJsonObject['frames'])):
        keypointsJsonObject = {
            "frameNum":  fidx,
            "keypoints3D": [],
            "keypoints": [],
            "heel3D": [
                {
                "name": "LeftHeel",
                "x": mediapipeJsonObject['frames'][fidx]['keypoints3D'][g_mediapipeName_idx_map['left_heel']]['x'],
                "y":mediapipeJsonObject['frames'][fidx]['keypoints3D'][g_mediapipeName_idx_map['left_heel']]['y'],
                "z":mediapipeJsonObject['frames'][fidx]['keypoints3D'][g_mediapipeName_idx_map['left_heel']]['z'],
                "score":mediapipeJsonObject['frames'][fidx]['keypoints3D'][g_mediapipeName_idx_map['left_heel']]['score']
                }, 
                {
                "name": "RightHeel",
                "x": mediapipeJsonObject['frames'][fidx]['keypoints3D'][g_mediapipeName_idx_map['right_heel']]['x'],
                "y":mediapipeJsonObject['frames'][fidx]['keypoints3D'][g_mediapipeName_idx_map['right_heel']]['y'],
                "z":mediapipeJsonObject['frames'][fidx]['keypoints3D'][g_mediapipeName_idx_map['right_heel']]['z'],
                "score":mediapipeJsonObject['frames'][fidx]['keypoints3D'][g_mediapipeName_idx_map['right_heel']]['score']
                },
            ]
        }
        mediapipeToMixamoHip(keypointsJsonObject, mediapipeJsonObject['frames'], fidx)
        mediapipeToMixamoSpine(keypointsJsonObject, mediapipeJsonObject['frames'], fidx)
        mediapipeToMixamoNeck(keypointsJsonObject, mediapipeJsonObject['frames'], fidx)
        mediapipeToMixamoHead(keypointsJsonObject, mediapipeJsonObject['frames'], fidx)
        mediapipeToMixamoAll(keypointsJsonObject, mediapipeJsonObject['frames'], fidx)
        jsonObject["frames"].append(keypointsJsonObject)
    return jsonObject

## **Open All Mediapipe Json file in /mediapipe_pose_to_json/output**

In [10]:
target_pattern = r"mediapipe_pose_to_json/output/*.json"
file_list = glob.glob(target_pattern)
file_list = [file.replace('\\', '/') for file in file_list]
for file in file_list:
    json_object = None
    with open(file) as f:
        json_object = json.load(f)
    mixamoJsonObject = mediapipeToMixamo(json_object)
    with open('./output/' + ntpath.basename(file) + '_mixamo.json', 'w') as f:
        json_string = json.dump(mixamoJsonObject, f, indent=2)    